# Assignment 1

## Question `2` (Decision Trees)

| | |
|-|-|
| Course | Statistical Methods in AI |
| Release Date | `19.01.2023` |
| Due Date | `29.01.2023` |

This assignment will have you working and experimenting with decision trees. Initially, you will be required to implement a decision tree classifier by choosing thresholds based on various impurity measures and reporting the scores. Later, you can experiment with the `scikit-learn` implementation of decision trees, and how various other parameters can be leveraged for better performance.

The dataset is a very simple one, the [banknote authentication dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication). It has 5 columns, the first 4 are the features, and the last one is the class label. The features are the variance, skewness, curtosis and entropy of the [wavelet transformed](https://en.wikipedia.org/wiki/Wavelet_transform) image of the banknote. The class label is 1 if the banknote is authentic, and 0 if it is forged. The data is present in `bankAuth.txt`. There are a total of 1372 samples in the dataset.

### Imports

In [12]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import *

# additional imports if necessary

### Impurity Measures

Decision trees are only as good as the impurity measure used to choose the best split. In this section, you will be required to implement the following impurity measures and use them to build a decision tree classifier.

1. Gini Index
2. Entropy
3. Misclassification Error
4. Log Loss

Write functions that calculate the impurity measures for a given set of labels. The functions should take in a list of labels and return the impurity measure.

In [13]:
def _gini(y):
  w = np.array(y)
  uniqw, inverse = np.unique(w, return_inverse=True)
  hist = np.bincount(inverse)
  ps = hist / len(y)
  return 1-np.sum([p ** 2 for p in ps if p>0])
      
# Function to perform training with entropy.
def _entropy(y):
  w = np.array(y)
  uniqw, inverse = np.unique(w, return_inverse=True)
  hist = np.bincount(inverse)
  ps = hist / len(y)
  return -np.sum([p * np.log(p) for p in ps if p>0])

def _mce(y):
  w = np.array(y)
  uniqw, inverse = np.unique(w, return_inverse=True)
  hist = np.bincount(inverse)
  ps = hist / len(y)
  return 1-np.max([(p , 1-p) for p in ps if p>0])


In [14]:
# your code here]

### Decision Tree

Fit a decision tree using any one of the above impurity measures with a depth of 3. This means you will have eight leaf nodes and seven internal nodes. Report the threshold values at each internal node and the impurity measure at the final leaf node with the label. Also report the accuracy of the classifier on the training and test data (instructions for splitting the data will be given in the end).

In [15]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.value = value
        self.right = right
        
    def is_leaf_node(self):
        return self.value is not None

In [16]:
# your code here
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.depth = None
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        print("Fitting the Data : ")
        self.root = self.grow_tree(X, y)

    def grow_tree(self, X, y, depth=0):
        nsamples, n_feats = X.shape
        yy = np.unique(y)
        n_labels = len(yy)

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or nsamples<self.min_samples_split):
            leaf_value = Counter(y).most_common(1)[0][0]
            # print("ggg",depth,self.max_depth,n_labels,n_samples,self.min_samples_split)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self.splitUtil(X, y, feat_idxs,depth)
        print("Best feature : ",best_feature)
        print("Best Threshold",best_thresh)
        # create child nodes
        left_idxs = np.argwhere(X[:, best_feature] <= best_thresh).flatten()
        right_idxs = np.argwhere(X[:, best_feature] > best_thresh).flatten()
        left = self.grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self.grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def splitUtil(self, X, y, feat_idxs,d):
        best_gain = -1
        temp = 0
        split_idx = None
        split_threshold = None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            X_col = X_column
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self.information_gain(y, X_column, thr,best_gain)
                temp = gain

                if gain > best_gain:
                    best_gain = gain
                    temp = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def information_gain(self, y, X_column, threshold,best_gain):
        parent_entropy = _mce(y)

        # create children
        left_idxs = np.argwhere(X_column <= threshold).flatten()
        right_idxs = np.argwhere(X_column > threshold).flatten()

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = _mce(y[left_idxs]), _mce(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def predict(self, X):
        return np.array([self.traverse(x, self.root,0) for x in X])

    def traverse(self, x, node,ht):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self.traverse(x, node.left,ht)
        else:
            return self.traverse(x, node.right,ht)

### `sklearn` Decision Tree Experiments

1. Scikit-learn has two decision tree implementations: [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and [`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html). 

When would you use one over the other? What would you use in the case of the banknote authentication dataset? Explain the changes that need to be made in the dataset to use the other implementation.

2. Fit a decision tree to the training set. Change various parameters and compare them to one another. Mainly try and experiment with the `criterion`, `max_depth` and `min_samples_split` parameters. Report the accuracy on the training and test set for each of the experiments while varying the parameters for comparison purposes.

3. Plot your trees !! (optional) (for visualization)

```python
from sklearn.tree import plot_tree

def plotTree(tree):
    """
    tree: Tree instance that is the result of fitting a DecisionTreeClassifier
          or a DecisionTreeRegressor.
    """
    plt.figure(figsize=(30,20))
    plot_tree(tree, filled=True, rounded=True,
                  class_names=['forged', 'authentic'],
                  feature_names=['var', 'skew', 'curt', 'ent'])
    plt.show()
    return None
```

DecisionTreeClassifier is used for classification tasks, where the target variable is categorical. DecisionTreeRegressor is used for regression tasks, where the target variable is continuous.

For the banknote authentication dataset, since the target variable is whether the note is authentic or not (categorical), you would use DecisionTreeClassifier.

No changes need to be made to the dataset to use the other implementation, just change the classifier used (e.g. from DecisionTreeClassifier to DecisionTreeRegressor).

In [17]:
# your code here

### Load Data

The data has been loaded onto a Pandas DataFrame. Try to get an initial feel for the data by using functions like `describe()`, `info()`, or maybe try to plot the data to check for any patterns.

Note: To obtain the data from the UCI website, `wget` can be used followed by shuffling the samples using `shuf` and adding a header for easier reading via `pandas`. It is not necessary to view the data in a DataFrame and can be directly loaded onto NumPy as convenient.

In [18]:
data = pd.read_csv('bankAuth.txt')

In [19]:
# your code here
def importdata():
    balance_data = pd.read_csv('bankAuth.txt' , names=["variance", "skewness", "curtosis" , "entropy","target"])
      
    # Printing the dataswet shape
    print ("Dataset Length: ", len(balance_data))
    print ("Dataset Shape: ", balance_data.shape)
      
    # Printing the dataset obseravtions
    print ("Dataset: ")
    print(balance_data.head())
    return balance_data

data = importdata()

Dataset Length:  1372
Dataset Shape:  (1372, 5)
Dataset: 
   variance  skewness  curtosis  entropy  target
0   3.62160    8.6661   -2.8073 -0.44699       0
1   4.54590    8.1674   -2.4586 -1.46210       0
2   3.86600   -2.6383    1.9242  0.10645       0
3   3.45660    9.5228   -4.0112 -3.59440       0
4   0.32924   -4.4552    4.5718 -0.98880       0


### Splitting the Data

It is a good practice to split the data into training and test sets. This is to ensure that the model is not overfitting to the training data. The test set is used to evaluate the performance of the model on unseen data. The test set is not used to train the model in any way. The test set is only used to evaluate the performance of the model. You may use the `train_test_split` function from `sklearn.model_selection` to split the data into training and test sets.

It is a good idea to move your data to NumPy arrays now as it will make computing easier.

In [20]:
# your code here
# Function to split the dataset
def splitdataset(balance_data):
  
    # Separating the target variable
    X = balance_data.values[:, :-1]
    Y = balance_data.values[:, 4]
    # Splitting the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.3, random_state = 100)
      
    return X, Y, X_train, X_test, y_train, y_test
X, Y, X_train, X_test, y_train, y_test = splitdataset(data)
print(X_train.shape)
print(y_train.shape)

(960, 4)
(960,)


### Denouement

Use this place to report all comparisons and wrap up the calls to the functions written above.

In [21]:
# your code here
clf = DecisionTree(max_depth=3)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc*100)

Best feature :  0
Best Threshold 0.26877
Best feature :  1
Best Threshold 7.5032
Best feature :  2
Best Threshold -2.5237
Best feature :  0
Best Threshold -5.3857
Best feature :  2
Best Threshold -4.4738
Best feature :  0
Best Threshold 2.3917
Best feature :  1
Best Threshold -4.8835
93.93203883495146


Entropy : 95.63106796116504

Gini : 94.1747572815534

Misclassification Error : 93.93203883495146
